In [1]:
%pip install datasets
%pip install transformers

     -------------------------------------- 451.6/451.6 kB 5.7 MB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 6.3 MB/s eta 0:00:00
     ------------------------------------- 182.1/182.1 kB 11.5 MB/s eta 0:00:00
     --------------------------------------- 11.0/11.0 MB 43.7 MB/s eta 0:00:00
     ------------------------------------- 324.3/324.3 kB 19.6 MB/s eta 0:00:00
     -------------------------------------- 155.4/155.4 kB 9.1 MB/s eta 0:00:00
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     -------------------------------------- 139.5/139.5 kB 8.1 MB/s eta 0:00:00
     -------------------------------------- 132.0/132.0 kB 8.1 MB/s eta 0:00:00
     --------------------------------------- 20.0/20.0 MB 46.7 MB/s eta 0:00:00
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ---------------------------------------- 56.8/56.8 kB 3.1 MB/s eta 0:00:00
     ---------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     ---------------------------------------- 5.5/5.5 MB 21.9 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 42.2 MB/s eta 0:00:00
     ------------------------------------- 267.7/267.7 kB 17.2 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: C:\Users\Brian\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from datasets import Dataset, load_dataset, load_metric
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
import json

c:\Users\Brian\anaconda3\envs\nattyLP\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Download datasets from hugging face

missing children's book dataset

In [2]:
#ag_dataset_train = load_dataset('ag_news', split='train')
#ag_dataset_test = load_dataset('ag_news', split='test')
#imdb_dataset_train = load_dataset('imdb', split='train')
#imdb_dataset_test = load_dataset('imdb', split='test')
rt_dataset_train = load_dataset('rotten_tomatoes', split='train')
rt_dataset_test = load_dataset('rotten_tomatoes', split='test')

Found cached dataset rotten_tomatoes (C:/Users/Brian/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)
Found cached dataset rotten_tomatoes (C:/Users/Brian/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


Building tokenizer

In [3]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_function(examples):
	return tokenizer(examples["text"], padding="max_length", truncation=True)

#tk_imdb_dataset_train = imdb_dataset_train.map(tokenize_function, batched=True)
#tk_imdb_dataset_test = imdb_dataset_test.map(tokenize_function, batched=True)
#tk_ag_dataset_train = ag_dataset_train.map(tokenize_function, batched = True)
#tk_ag_dataset_test = ag_dataset_test.map(tokenize_function, batched=True)
tk_rt_dataset_train = rt_dataset_train.map(tokenize_function, batched=True)
tk_rt_dataset_test = rt_dataset_test.map(tokenize_function, batched=True)

building tokenizer...


Loading cached processed dataset at C:/Users/Brian/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46\cache-920be128161bdecb.arrow
Loading cached processed dataset at C:/Users/Brian/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46\cache-a5c077e2665a9fb5.arrow


Rotten Tomatoes Dataset

In [4]:
tk_rt_dataset_train = tk_rt_dataset_train.shuffle(seed=42)
data = tk_rt_dataset_train.train_test_split(.1)
tk_rt_train = data['train']
tk_rt_eval = data['test']

Loading cached shuffled indices for dataset at C:/Users/Brian/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46\cache-3b128816ae75adce.arrow


In [5]:
tk_rt_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 7677
})

In [6]:
tk_rt_eval

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 853
})

In [7]:
np.unique(tk_rt_eval['label'], return_counts=True)

(array([0, 1]), array([409, 444], dtype=int64))

In [8]:
tk_rt_dataset_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1066
})

In [9]:
np.unique(tk_rt_train['label'], return_counts=True)

(array([0, 1]), array([3856, 3821], dtype=int64))

In [10]:
np.unique(tk_rt_dataset_test['label'], return_counts=True)

(array([0, 1]), array([533, 533], dtype=int64))

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 1e-6#, #defaults to 5e-5
	# report_to="wandb"
	)

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	print("Labels:", labels)
	print("Predictions:", predictions)
	# print("Logits:", logits)
	# print("Sizes:", labels.shape, predictions.shape, logits.shape)
	# exit()
	# label = np.array([random.randint(0,1) for _ in range(26112)])
	# preds = np.array([random.randint(0,1) for _ in range(26112)])
	# logits = np.array([[random.random() for _ in range(2)] for _ in range(26112)])
	result = metric.compute(predictions=predictions, references=labels)
	return result

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [ ]:
#model.cuda()
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7677
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2880
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster tha

Step,Training Loss,Validation Loss,Accuracy
100,0.702400,0.687396,0.526377
200,0.686100,0.679397,0.549824
300,0.676400,0.664804,0.730363
400,0.659000,0.643372,0.763189
500,0.629800,0.609105,0.787808
600,0.606800,0.573596,0.796014
700,0.561500,0.544438,0.787808
800,0.529400,0.515096,0.800703
900,0.511900,0.496395,0.797186
1000,0.468100,0.475928,0.805393


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


...

Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out/checkpoint-2500 (score: 0.4291459619998932).


TrainOutput(global_step=2880, training_loss=0.4882192889849345, metrics={'train_runtime': 1579.209, 'train_samples_per_second': 14.584, 'train_steps_per_second': 1.824, 'total_flos': 3050856658446336.0, 'train_loss': 0.4882192889849345, 'epoch': 3.0})

In [ ]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.0932975 ,  0.9531956 ],
       [-1.187784  ,  1.060396  ],
       [ 0.9511346 , -0.61736566],
       ...,
       [ 0.7267991 , -0.6463394 ],
       [ 0.67282224, -0.39806432],
       [ 1.1171148 , -0.9992569 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.41931256651878357, 'test_accuracy': 0.8142589118198874, 'test_runtime': 18.924, 'test_samples_per_second': 56.33, 'test_steps_per_second': 7.081})

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 1e-5#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

Step,Training Loss,Validation Loss,Accuracy
100,0.658000,0.542995,0.787808
200,0.498300,0.434994,0.806565
300,0.400800,0.424976,0.813599
400,0.394300,0.462440,0.815944
500,0.382200,0.436209,0.818288
600,0.444600,0.396142,0.826495
700,0.379700,0.434008,0.825322
800,0.403700,0.387679,0.834701
900,0.378200,0.384142,0.837046
1000,0.293600,0.431767,0.834701


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


...

Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out/checkpoint-1000 (score: 0.4317670464515686).


TrainOutput(global_step=2880, training_loss=0.3180938654475742, metrics={'train_runtime': 1560.8438, 'train_samples_per_second': 14.755, 'train_steps_per_second': 1.845, 'total_flos': 3050856658446336.0, 'train_loss': 0.3180938654475742, 'epoch': 3.0})

In [ ]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


Labels: [1 1 1 ... 0 0 0]
Predictions: [0 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[ 0.53838944, -0.4914009 ],
       [-2.0593567 ,  2.246587  ],
       [ 1.5731484 , -1.4177113 ],
       ...,
       [ 1.6457624 , -1.5862218 ],
       [ 1.644735  , -1.5781479 ],
       [ 1.7641137 , -1.8821074 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.4574543535709381, 'test_accuracy': 0.8273921200750469, 'test_runtime': 18.9561, 'test_samples_per_second': 56.235, 'test_steps_per_second': 7.069})

In [ ]:
#lr
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-5#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

Step,Training Loss,Validation Loss,Accuracy
100,0.585800,0.441214,0.804220
200,0.474000,0.461464,0.796014
300,0.434100,0.532264,0.807737
400,0.439600,0.455174,0.818288
500,0.406900,0.541480,0.790152
600,0.427400,0.417320,0.827667
700,0.430900,0.408914,0.834701
800,0.396500,0.429186,0.831184
900,0.393200,0.447984,0.819461
1000,0.283700,0.735651,0.791325


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


...

Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out/checkpoint-500 (score: 0.541479766368866).


TrainOutput(global_step=2880, training_loss=0.2563622521029578, metrics={'train_runtime': 1559.2727, 'train_samples_per_second': 14.77, 'train_steps_per_second': 1.847, 'total_flos': 3050856658446336.0, 'train_loss': 0.2563622521029578, 'epoch': 3.0})

In [ ]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.1778222 ,  0.80427676],
       [-2.2168427 ,  1.7851115 ],
       [ 1.4298048 , -1.5598465 ],
       ...,
       [ 1.8058231 , -1.9690747 ],
       [ 1.7649186 , -1.8708811 ],
       [ 1.913838  , -2.143399  ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.5337198972702026, 'test_accuracy': 0.7945590994371482, 'test_runtime': 18.9044, 'test_samples_per_second': 56.389, 'test_steps_per_second': 7.088})

In [ ]:
#dropout
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2,
                                                           dropout = 0.05)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.05,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when i

Step,Training Loss,Validation Loss,Accuracy
100,0.684400,0.655699,0.703400
200,0.576400,0.496962,0.779601
300,0.430600,0.439654,0.807737
400,0.416400,0.452392,0.798359
500,0.411500,0.421695,0.815944
600,0.439700,0.410477,0.813599
700,0.387800,0.425698,0.807737
800,0.412200,0.410749,0.815944
900,0.387400,0.397856,0.828839
1000,0.319800,0.407920,0.826495


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


...

Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


Labels: [0 0 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 1
 1 0 0 1 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 0 0 0
 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 0 1 1
 0 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 0
 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0
 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0
 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 1
 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 1 0
 0 1 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 1
 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 0
 1 1 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1
 1 0 0 1 0 0 1 0 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out/checkpoint-1000 (score: 0.40791967511177063).


TrainOutput(global_step=2880, training_loss=0.35244850715001425, metrics={'train_runtime': 1558.5969, 'train_samples_per_second': 14.777, 'train_steps_per_second': 1.848, 'total_flos': 3050856658446336.0, 'train_loss': 0.35244850715001425, 'epoch': 3.0})

In [ ]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-0.35581848,  0.13269733],
       [-1.8285384 ,  1.5150595 ],
       [ 1.1418566 , -1.2194825 ],
       ...,
       [ 1.153171  , -1.3300481 ],
       [ 1.0371413 , -1.2486043 ],
       [ 1.4466611 , -1.69973   ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.4178270697593689, 'test_accuracy': 0.8255159474671669, 'test_runtime': 18.9491, 'test_samples_per_second': 56.256, 'test_steps_per_second': 7.072})

In [13]:
#dropout
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2,
                                                           dropout = 0.15)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

  0%|          | 0/2880 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.673, 'learning_rate': 4.8263888888888895e-06, 'epoch': 0.1}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1
 0 1 1 0 0 0 0 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-1000 (score: 0.351763516664505).


{'train_runtime': 562.5553, 'train_samples_per_second': 40.94, 'train_steps_per_second': 5.119, 'train_loss': 0.3746844655937619, 'epoch': 3.0}


TrainOutput(global_step=2880, training_loss=0.3746844655937619, metrics={'train_runtime': 562.5553, 'train_samples_per_second': 40.94, 'train_steps_per_second': 5.119, 'train_loss': 0.3746844655937619, 'epoch': 3.0})

In [14]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-0.68509483,  0.8572299 ],
       [-1.5349523 ,  1.8773688 ],
       [ 1.1437578 , -1.2153604 ],
       ...,
       [ 1.2533257 , -1.3539706 ],
       [ 1.3236287 , -1.3861327 ],
       [ 1.4339617 , -1.5902905 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.40804266929626465, 'test_accuracy': 0.8311444652908068, 'test_runtime': 6.2425, 'test_samples_per_second': 170.764, 'test_steps_per_second': 21.466})

In [15]:
#dropout
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2,
                                                           dropout = 0.2)
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 16,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.2,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

  0%|          | 0/2880 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.685, 'learning_rate': 4.8263888888888895e-06, 'epoch': 0.1}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1
 0 1 1 0 0 0 0 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-1000 (score: 0.3550567328929901).


{'train_runtime': 583.7813, 'train_samples_per_second': 39.451, 'train_steps_per_second': 4.933, 'train_loss': 0.39011315306027733, 'epoch': 3.0}


TrainOutput(global_step=2880, training_loss=0.39011315306027733, metrics={'train_runtime': 583.7813, 'train_samples_per_second': 39.451, 'train_steps_per_second': 4.933, 'train_loss': 0.39011315306027733, 'epoch': 3.0})

In [16]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.0947759,  0.8179082],
       [-1.801247 ,  1.4911013],
       [ 1.2094843, -1.3031027],
       ...,
       [ 1.2165945, -1.4020638],
       [ 1.0104544, -1.2365797],
       [ 1.3999884, -1.6904794]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.4085255563259125, 'test_accuracy': 0.8330206378986866, 'test_runtime': 6.5156, 'test_samples_per_second': 163.607, 'test_steps_per_second': 20.566})

In [17]:
#batch size = 8
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2
                                                           )
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 100,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	per_device_train_batch_size = 8,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

  0%|          | 0/2880 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.6815, 'learning_rate': 4.8263888888888895e-06, 'epoch': 0.1}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1
 0 1 1 0 0 0 0 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-1000 (score: 0.3514048755168915).


{'train_runtime': 585.4367, 'train_samples_per_second': 39.34, 'train_steps_per_second': 4.919, 'train_loss': 0.36518810987472533, 'epoch': 3.0}


TrainOutput(global_step=2880, training_loss=0.36518810987472533, metrics={'train_runtime': 585.4367, 'train_samples_per_second': 39.34, 'train_steps_per_second': 4.919, 'train_loss': 0.36518810987472533, 'epoch': 3.0})

In [18]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.0888566,  0.7878324],
       [-1.8861798,  1.5596639],
       [ 1.2399204, -1.3750496],
       ...,
       [ 1.2463005, -1.435207 ],
       [ 1.0889246, -1.3320307],
       [ 1.4419832, -1.7535889]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.4118333160877228, 'test_accuracy': 0.8330206378986866, 'test_runtime': 6.423, 'test_samples_per_second': 165.965, 'test_steps_per_second': 20.862})

In [19]:
#warm up
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2
                                                           )
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	warmup_steps = 0,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 8,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

  0%|          | 0/2880 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.6815, 'learning_rate': 4.8263888888888895e-06, 'epoch': 0.1}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1
 0 1 1 0 0 0 0 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-1000 (score: 0.3514043986797333).


{'train_runtime': 564.9422, 'train_samples_per_second': 40.767, 'train_steps_per_second': 5.098, 'train_loss': 0.3651890589131249, 'epoch': 3.0}


TrainOutput(global_step=2880, training_loss=0.3651890589131249, metrics={'train_runtime': 564.9422, 'train_samples_per_second': 40.767, 'train_steps_per_second': 5.098, 'train_loss': 0.3651890589131249, 'epoch': 3.0})

In [20]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.0888587,  0.7878388],
       [-1.8861796,  1.5596675],
       [ 1.2399217, -1.3750522],
       ...,
       [ 1.2462963, -1.435207 ],
       [ 1.088925 , -1.3320287],
       [ 1.441981 , -1.7535884]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.41183292865753174, 'test_accuracy': 0.8330206378986866, 'test_runtime': 6.1643, 'test_samples_per_second': 172.931, 'test_steps_per_second': 21.738})

In [21]:
#warm up
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2
                                                           )
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	warmup_steps = 200,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 8,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

  0%|          | 0/2880 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.6924, 'learning_rate': 2.5e-06, 'epoch': 0.1}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1
 0 1 1 0 0 0 0 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-1000 (score: 0.35987579822540283).


{'train_runtime': 556.3787, 'train_samples_per_second': 41.394, 'train_steps_per_second': 5.176, 'train_loss': 0.3735758754942152, 'epoch': 3.0}


TrainOutput(global_step=2880, training_loss=0.3735758754942152, metrics={'train_runtime': 556.3787, 'train_samples_per_second': 41.394, 'train_steps_per_second': 5.176, 'train_loss': 0.3735758754942152, 'epoch': 3.0})

In [22]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.0566591 ,  0.76107085],
       [-1.8355925 ,  1.5338559 ],
       [ 1.2475003 , -1.3897383 ],
       ...,
       [ 1.2408562 , -1.438453  ],
       [ 1.0775315 , -1.3287396 ],
       [ 1.4101962 , -1.7121004 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.41689759492874146, 'test_accuracy': 0.8302063789868668, 'test_runtime': 5.9897, 'test_samples_per_second': 177.974, 'test_steps_per_second': 22.372})

In [23]:
#warm up
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2
                                                           )
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	warmup_steps = 400,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	#per_device_train_batch_size = 8,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

  0%|          | 0/2880 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.693, 'learning_rate': 1.25e-06, 'epoch': 0.1}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 1 0 1
 0 1 0 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1
 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0
 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1
 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1
 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0
 0 0 1 1 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1
 0 1 1 0 0 0 0 1 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-1000 (score: 0.36121758818626404).


{'train_runtime': 563.0598, 'train_samples_per_second': 40.903, 'train_steps_per_second': 5.115, 'train_loss': 0.3831281489796109, 'epoch': 3.0}


TrainOutput(global_step=2880, training_loss=0.3831281489796109, metrics={'train_runtime': 563.0598, 'train_samples_per_second': 40.903, 'train_steps_per_second': 5.115, 'train_loss': 0.3831281489796109, 'epoch': 3.0})

In [24]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-1.1060172,  0.8125644],
       [-1.7522683,  1.478715 ],
       [ 1.2237699, -1.377037 ],
       ...,
       [ 1.2404479, -1.431231 ],
       [ 1.0824763, -1.3416356],
       [ 1.389689 , -1.6621333]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.4144597053527832, 'test_accuracy': 0.8330206378986866, 'test_runtime': 6.2329, 'test_samples_per_second': 171.029, 'test_steps_per_second': 21.499})

In [15]:
#batch size = 32
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2
                                                           )
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 400,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	per_device_train_batch_size = 2,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

loading configuration file config.json from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.23.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\Brian/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

  0%|          | 0/11517 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.689, 'learning_rate': 4.956585916471304e-06, 'epoch': 0.03}


  0%|          | 0/107 [00:00<?, ?it/s]

...

Labels: [0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0
 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1
 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1
 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0
 1 1 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1
 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1
 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0
 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1
 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 1
 1 0 0 1 0 0 1 0 

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.6803, 'learning_rate': 3.6107493270817055e-06, 'epoch': 0.83}


  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0
 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1
 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1
 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0
 1 1 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1
 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1
 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0
 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1
 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 1
 1 0 0 1 0 0 1 0 

In [ ]:
trainer.predict(tk_rt_dataset_test)

In [12]:
#batch size = 4
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", 
                                                           num_labels = 2
                                                           )
model.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/nlp/model_out", 
	evaluation_strategy="steps", 
	num_train_epochs = 3,
  eval_steps = 100,
	#warmup_steps = 400,
	logging_steps = 100,
  #gradient_accumulation_steps=4,
	# evaluate_during_training = True, #outdated parameter option, do not use
	per_device_train_batch_size = 4,
	save_steps = 500,
	load_best_model_at_end = True,
	learning_rate = 5e-6#, #defaults to 5e-5
	# report_to="wandb"
	)
trainer = Trainer(
	model=model,
	args = training_args,
	train_dataset = tk_rt_train,
	eval_dataset = tk_rt_eval,
	tokenizer=tokenizer,
	compute_metrics=compute_metrics
	)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

  0%|          | 0/5760 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 853
  Batch size = 8


{'loss': 0.6932, 'learning_rate': 4.9131944444444445e-06, 'epoch': 0.05}


  0%|          | 0/107 [00:00<?, ?it/s]

...

  0%|          | 0/107 [00:00<?, ?it/s]

Labels: [0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 0 0
 0 1 1 1 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 1 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1
 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 1 1
 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1
 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 1 0
 1 1 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 1
 0 1 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 0 1
 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0
 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 1 1 1
 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0
 0 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 0 1 0 0 1
 1 0 0 1 0 0 1 0 



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/My Drive/nlp/model_out\checkpoint-500 (score: 0.4561321437358856).


{'train_runtime': 769.7203, 'train_samples_per_second': 29.921, 'train_steps_per_second': 7.483, 'train_loss': 0.4161901169353061, 'epoch': 3.0}


TrainOutput(global_step=5760, training_loss=0.4161901169353061, metrics={'train_runtime': 769.7203, 'train_samples_per_second': 29.921, 'train_steps_per_second': 7.483, 'train_loss': 0.4161901169353061, 'epoch': 3.0})

In [13]:
trainer.predict(tk_rt_dataset_test)

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1066
  Batch size = 8


  0%|          | 0/134 [00:00<?, ?it/s]

Labels: [1 1 1 ... 0 0 0]
Predictions: [1 1 0 ... 0 0 0]


PredictionOutput(predictions=array([[-0.75089496,  0.6298743 ],
       [-1.4982893 ,  1.1012908 ],
       [ 1.3282588 , -0.9242535 ],
       ...,
       [ 0.99772334, -0.6115895 ],
       [ 1.0559719 , -0.7777182 ],
       [ 1.3929151 , -0.9941501 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0], dtype=int64), metrics={'test_loss': 0.4463183283805847, 'test_accuracy': 0.801125703564728, 'test_runtime': 6.0076, 'test_samples_per_second': 177.442, 'test_steps_per_second': 22.305})